# Demonstrasjon av filformatet Apache Markdown med Python Pandas
Brønnøysundregistrene er en av mange aktører som tilbyr åpne data, både gjennom API-er og gjennom nedlasting av filer.
Du kan for eksempel laste ned hele eller deler av Enhetsregisteret som en JSON-fil eller Excel-fil. Det første formatet
er populært for de som skal bruke dataene ved hjelp av et programmeringsspråk, mens det siste er populært for alle andre.

Begge formatene har fordeler og ulemper. Fordelen med Excel-formatet er selvsagt at det gjør det veldig lett å bruke
dataene med regneark-program som Microsoft Excel. Og siden det er et så populært program, er det også mange andre
verktøy som støtter det formatet.

Stadig flere bruker Python og Pandas for å jobbe med data, både for å forberede dataene for analyse, og for selve analysen. 
Og mange bruker et "notebook"-verktøy, for å gjøre dette, slik som jeg har gjort her, gjennom en Jupyter Notebook.

En av grunnene til at det er lett å bruke Pandas er at det støtter Excel-formatet (og JSON-formatet), slik at det er veldig
lett å jobbe med data som finnes allerede i Excel, og å lagre resultatet av det du har gjort som Excel for å dele det med
andre.

Men Pandas støtter også formatet Apache Parquet. Nedenfor skal jeg demonstrere noen av fordelene med det formatet.

Men først, vi må få på plass de verktøyene vi trenger, og det er egentlig bare ett: Pandas

In [2]:
%pip install pandas

  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached numpy-1.21.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install openpyxl

  Using cached openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

# Laste ned et passende utvalg av Enhetsregisteret i Excel-format
Vær oppmerksom på at du enten må laste ned _hele_ registeret, eller gjøre et søk så du får maksimum 10.000 enheter i resultatet, for med mindre du laster ned hele registeret, får du uansett maks 10.000 enheter.

Istedenfor å bruke Python til å laste ned fila bruker jeg kommandolinjeverktøyet curl. Men det er også mulig å bare laste det ned manuelt fra siden og lagre det i samme mappe som Jypyter-fila.

Merk at det er en stor fil (ca 200 MB) så det kan ta noen minutter.

In [4]:
%curl 'https://data.brreg.no/enhetsregisteret/api/enheter/lastned/regneark' -X GET \
    -H 'Accept: application/vnd.brreg.enhetsregisteret.enhet+vnd.openxmlformats-officedocument.spreadsheetml.sheet;charset=UTF-8' -J -O

('er_utvalg.xlsx', <http.client.HTTPMessage at 0x7ff7dad5ebb0>)

## Lese Excel-fila med Pandas
Vi starter med å bruke Pandas til å lese fila, uten å fortelle noe mer detaljer. For å få dokumentert tiden det tar, bruker vi Jypyter-funksjonen 'timeit'. Lurt å ha noe annet å holde på med når dette pågår ... Å svare på epost eller skrive julekort mens maskinen din leser inn hele enhetsregisteret, er eksempel på multitasking som faktisk sparer tid.

In [5]:
%%timeit
enheter = pd.read_excel('enheter_alle.xlsx')

11min 18s ± 28.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Resultatet viser at det tar i snitt drøyt 11 minutter å laste inn fila:
```11min 18s ± 28.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)```

La oss se nærmere på resultatet i form av minnebruk og hvordan Pandas har tolket de ulike feltene

In [6]:
enheter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 43 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   Organisasjonsnummer                           1048575 non-null  int64  
 1   Navn                                          1048575 non-null  object 
 2   Organisasjonsform.kode                        1048575 non-null  object 
 3   Organisasjonsform.beskrivelse                 1048575 non-null  object 
 4   Næringskode 1                                 991572 non-null   float64
 5   Næringskode 1.beskrivelse                     991572 non-null   object 
 6   Næringskode 2                                 40887 non-null    float64
 7   Næringskode 2.beskrivelse                     40887 non-null    object 
 8   Næringskode 3                                 1542 non-null     float64
 9   Næringskode 3.beskrivelse          

Her ser vi at registeret bruker nesten 350 MB minne slik det er lest inn nå. I kolonnen Dtype ser vi at de fleste feltene har fått datatypen "object", som er den generelle datatypen Pandas bruker når den ikke vet bedre. Det er også den datatypen som bruker mest minne.

Når vi ser på tabellen ser vi at det er en rekke endringer vi burde gjøre. For det første er Organisasjonsnummer en identifikator, og ikke noe vi skal regne med, så det bør ikke være behandles som et tall (int64). Det er også en del datoer (de er ikke med i sammendraget her), og de blir også behandlet som "object", og dermed blir det vanskeligere å bruke Pandas funksjoner for å regne med datoer. 

## Lagre til en parquet-fil


In [8]:
%pip install pyarrow

  Using cached pyarrow-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
%%timeit
enheter.to_parquet('enheter_alle.parquet')

3.06 s ± 624 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


På min maskin tar det i snitt drøyt tre sekunder å lagre hele registeret til en parquet-fil:
```11min 18s ± 28.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)```

Det er også interessant å se at størrelsen er så forskjellig på de to filene:

In [10]:
%ll enheter_alle.*

-rw-r--r-- 1 wslstsk  57558674 Nov 27 19:15 enheter_alle.parquet
-rw-r--r-- 1 wslstsk 206038606 Nov 27 17:01 enheter_alle.xlsx


parquet-fila er nesten bare en fjerdedel av Excel-fila.

Lesing av parquet-fila går sjokkerende mye raskere enn å lese excel:

In [6]:
%%timeit
enheter = pd.read_parquet('enheter_alle.parquet')

1.68 s ± 296 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Med timeit ser vi at det i stnitt tar under _to_ sekunder å lese hele registeret fra parquet. Mot 11 minutter for Excel.
```1.68 s ± 296 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)```

Men har vi da fått med de samme dataene? La oss se:

In [7]:
enheter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 43 columns):
 #   Column                                        Non-Null Count    Dtype  
---  ------                                        --------------    -----  
 0   Organisasjonsnummer                           1048575 non-null  int64  
 1   Navn                                          1048575 non-null  object 
 2   Organisasjonsform.kode                        1048575 non-null  object 
 3   Organisasjonsform.beskrivelse                 1048575 non-null  object 
 4   Næringskode 1                                 991572 non-null   float64
 5   Næringskode 1.beskrivelse                     991572 non-null   object 
 6   Næringskode 2                                 40887 non-null    float64
 7   Næringskode 2.beskrivelse                     40887 non-null    object 
 8   Næringskode 3                                 1542 non-null     float64
 9   Næringskode 3.beskrivelse          

Det er like mange kolonner, og like mye minne brukes, så det virker som vi har fått med de samme dataene.

## Konklusjon: Stor forskjell på filstørrelse og ENORM i lesehastighet
Uten å gjøre noen forsøk på å optimalisere dataene, ser vi likevel at filstørrelsen blir redusert til en fjerdedel, men aller viktigst: Det tar _380_ ganger så lang tid å lese de samme dataene fra en Excel-fil, som fra en parquet-fil!!

In [11]:
11*60/1.7

388.2352941176471

## Optimalisering av dataene -- hvordan påvirker det resultatet?
La oss sette riktige datatyper på kolonnene, det trengs uansett for å kunne analysere dataene effektivt.

In [4]:
import numpy as np

In [24]:
enheter = pd.read_parquet('enheter_alle.parquet')

In [25]:
enheter = enheter.astype(
    dtype={
        'Organisasjonsnummer': str,
        'Navn': str,
        'Organisasjonsform.kode': 'category',
        'Organisasjonsform.beskrivelse': 'category',
        'Næringskode 1': str,
        'Næringskode 1.beskrivelse': str,
        'Næringskode 2': str,
        'Næringskode 2.beskrivelse': str,
        'Næringskode 3': str,
        'Næringskode 3.beskrivelse': str,
        'Hjelpeenhetskode': 'category',
        'Hjelpeenhetskode.beskrivelse': 'category',
        'Antall ansatte': np.int16,
        'Hjemmeside': str,
        'Postadresse.adresse': str,
        'Postadresse.poststed': 'category',
        'Postadresse.postnummer': 'category',
        'Postadresse.kommune': 'category',
        'Postadresse.kommunenummer': 'category',
        'Postadresse.land': 'category',
        'Postadresse.landkode': 'category',
        'Forretningsadresse.adresse': str,
        'Forretningsadresse.poststed': 'category',
        'Forretningsadresse.postnummer': 'category',
        'Forretningsadresse.kommune': 'category',
        'Forretningsadresse.kommunenummer': 'category',
        'Forretningsadresse.land': 'category',
        'Forretningsadresse.landkode': 'category',
        'Institusjonell sektorkode': 'category',
        'Institusjonell sektorkode.beskrivelse': 'category',
        'Siste innsendte årsregnskap': 'category', # klarte ikke konvertere til np.int16
        'Registreringsdato i Enhetsregisteret': np.datetime64, # str, # klarer ikke konvertere 'datetime64',
        'Stiftelsesdato': str, # klarte ikke å konvertere til datetime64 - 1550-12-31 00:00:00
        'FrivilligRegistrertIMvaregisteret': 'category',
        'Registrert i MVA-registeret': 'category',
        'Registrert i Frivillighetsregisteret': 'category',
        'Registrert i Foretaksregisteret': 'category',
        'Registrert i Stiftelsesregisteret': 'category',
        'Konkurs': 'category',
        'Under avvikling': 'category',
        'Under tvangsavvikling eller tvangsoppløsning': 'category',
        'Overordnet enhet i offentlig sektor': str,
        'Målform': 'category' }
)

In [26]:
enheter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 43 columns):
 #   Column                                        Non-Null Count    Dtype         
---  ------                                        --------------    -----         
 0   Organisasjonsnummer                           1048575 non-null  object        
 1   Navn                                          1048575 non-null  object        
 2   Organisasjonsform.kode                        1048575 non-null  category      
 3   Organisasjonsform.beskrivelse                 1048575 non-null  category      
 4   Næringskode 1                                 1048575 non-null  object        
 5   Næringskode 1.beskrivelse                     1048575 non-null  object        
 6   Næringskode 2                                 1048575 non-null  object        
 7   Næringskode 2.beskrivelse                     1048575 non-null  object        
 8   Næringskode 3                             

## Konklusjon: Mer enn halvert minnebruk!
Nå som vi har endret datatypen til mange av feltene, ser vi at minnebruken har falt betraktelig, nesten 200 MB (opprinnelig 340+ MB).

# Lagre den forbedrede varianten til parquet
Nå som vi har satt riktige datatyper på en rekke av kolonnene, kan vi lagre en ny parquet-fil, for sammenligningens skyld.

In [28]:
enheter.to_parquet('enheter_alle_forbedret.parquet')

Resultatet er -- litt overraskende -- at fila er blitt _litt_ større enn den var:

In [29]:
%ll *.parquet

-rw-r--r-- 1 wslstsk 57558674 Nov 27 19:15 enheter_alle.parquet
-rw-r--r-- 1 wslstsk 61147231 Nov 29 07:04 enheter_alle_forbedret.parquet


Spørsmålet er nå, hvis vi leser den nye fila, vil den automatisk få riktig datatyper da?

In [30]:
enheter2 = pd.read_parquet('enheter_alle_forbedret.parquet')

In [31]:
enheter2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 43 columns):
 #   Column                                        Non-Null Count    Dtype         
---  ------                                        --------------    -----         
 0   Organisasjonsnummer                           1048575 non-null  object        
 1   Navn                                          1048575 non-null  object        
 2   Organisasjonsform.kode                        1048575 non-null  category      
 3   Organisasjonsform.beskrivelse                 1048575 non-null  category      
 4   Næringskode 1                                 1048575 non-null  object        
 5   Næringskode 1.beskrivelse                     1048575 non-null  object        
 6   Næringskode 2                                 1048575 non-null  object        
 7   Næringskode 2.beskrivelse                     1048575 non-null  object        
 8   Næringskode 3                             

Minnebruken har økt en del (ca 50 MB), og det skyldes antageligvis at noen av kolonnene har endret fra 'category' til andre datatyper. Opprinnelig var det 28 'category', nå er det 21. Blant annet er alle postnumrene og kommunenumrene nå 'float64' istedenfor 'category'.

Det illustrerer at parquet ikke klarer å ta vare på alle faktaene om datasettet som vi har fortalt Pandas, men resultatet er uansett betraktelig bedre enn å lese Excel-fila. For det første er parquet-fila mye mindre, det går _utrolig_ mye fortere å lese den, den gir riktig datatype på mange flere av kolonnene, og resultatet er nesten 150 MB mindre enn ved innlesing fra Excel.

Siden vi ikke trenger enheter2 lenger, fjerner vi den.

In [32]:
del(enheter2)

## Kombinere med EHF-data
La oss se hva vi kan få ut av å se på data om EHF.

In [33]:
ehf = pd.read_parquet('/home/wslstsk/projects/nsg/sa1/statistikk/2021-11-26 mottakere ehf.parquet')

In [34]:
ehf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217816 entries, 69286 to 323900
Data columns (total 3 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   identifier                    217816 non-null  int64         
 1   regdate                       217816 non-null  datetime64[ns]
 2   PEPPOLBIS_3_0_BILLING_01_UBL  217816 non-null  category      
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 5.2 MB
